In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7'

In [ ]:
%run -p ../train_cnf_disentangle_rl_semisup.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/infocnf_cond_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_6_nsup4k_beta_0_1_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 6.0 --labeled_batch_size 450 --num_train_sup 4000 --beta 0.1
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_semisup.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWr

Load semi-sup datasets for CIFAR10


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0001 | Time 207.6194, Epoch Time 1791.4085(1791.4085), Bit/dim 7.0173(best: inf), Bit/dim Marginal 7.0170(best: inf), Xent 2.0832, Loss 8.0589, Error 0.7076(best: inf)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0110 | Time 13.3236(14.1643) | Bit/dim 6.9789(7.3713) | Xent 2.0808(2.0931) | Loss 18.1512(19.3662) | Error 0.6956(0.7078) Steps 544(523.42) | Grad Norm 8.7315(6.0047) | Total Time 0.00(0.00)
Iter 0120 | Time 14.1391(14.0686) | Bit/dim 6.9533(7.2658) | Xent 2.0297(2.0841) | Loss 17.8151(18.9725) | Error 0.6867(0.7049) Steps 514(526.49) | Grad Norm 3.5169(6.0685) | Total Time 0.00(0.00)
Iter 0130 | Time 14.0308(14.0655) | Bit/dim 6.9220(7.1794) | Xent 2.0018(2.0728) | Loss 17.7577(18.6714) | Error 0.6711(0.7004) Steps 556(531.58) | Grad Norm 4.4371(6.6061) | Total Time 0.00(0.00)
Iter 0140 | Time 13.9869(14.0150) | Bit/dim 6.8954(7.1065) | Xent 1.9783(2.0604) | Loss 17.7006(18.4358) | Error 0.6378(0.6931) Steps 568(538.48) | Grad Norm 4.1641(6.7680) | Total

...

Epoch 0005 | Time 116.2122, Epoch Time 1582.9599(1761.3564), Bit/dim 4.9808(best: 5.2121), Bit/dim Marginal 4.9364(best: 5.1716), Xent 1.9553, Loss 5.9584, Error 0.6862(best: 0.6651)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0520 | Time 14.5026(13.9420) | Bit/dim 4.8687(4.9862) | Xent 1.4995(1.6061) | Loss 13.0096(13.8519) | Error 0.5444(0.5544) Steps 574(554.05) | Grad Norm 11.4482(30.4564) | Total Time 0.00(0.00)
Iter 0530 | Time 14.0774(14.0270) | Bit/dim 4.8662(4.9619) | Xent 1.5169(1.5910) | Loss 12.8772(13.6307) | Error 0.5178(0.5502) Steps 544(555.90) | Grad Norm 22.2379(29.1879) | Total Time 0.00(0.00)
Iter 0540 | Time 14.6534(14.1128) | Bit/dim 4.8185(4.9394) | Xent 1.4606(1.5821) | Loss 12.7001(13.4679) | Error 0.5000(0.5472) Steps 562(556.59) | Grad Norm 20.5686(30.7609) | Total Time 0.00(0.00)
Iter 0550 | Time 14.3797(14.1435) | Bit/dim 4.8209(4.9213) | Xent 1.5525(1.5741) | Loss 12.8719(13.3265) | Error 0.5444(0.5457) Steps 568(554.61) | Grad Norm 42.8126

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0006 | Time 87.8730, Epoch Time 1645.1434(1757.8700), Bit/dim 4.8040(best: 4.9808), Bit/dim Marginal 4.7534(best: 4.9364), Xent 1.8668, Loss 5.7374, Error 0.6562(best: 0.6651)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0620 | Time 14.9768(14.6377) | Bit/dim 4.7569(4.7937) | Xent 1.6162(1.5221) | Loss 12.9090(13.4340) | Error 0.5822(0.5320) Steps 592(570.97) | Grad Norm 59.7294(32.9092) | Total Time 0.00(0.00)
Iter 0630 | Time 15.3457(14.7329) | Bit/dim 4.7178(4.7751) | Xent 1.5504(1.5237) | Loss 12.6993(13.2403) | Error 0.5289(0.5343) Steps 580(572.82) | Grad Norm 28.0964(32.9031) | Total Time 0.00(0.00)
Iter 0640 | Time 14.3982(14.7525) | Bit/dim 4.7263(4.7566) | Xent 1.5148(1.5119) | Loss 12.6345(13.0601) | Error 0.5511(0.5303) Steps 580(572.93) | Grad Norm 41.6279(31.6582) | Total Time 0.00(0.00)
Iter 0650 | Time 14.3027(14.7483) | Bit/dim 4.6558(4.7375) | Xent 1.4236(1.4950) | Loss 12.5085(12.9225) | Error 0.4978(0.5229) Steps 610(576.41) | Grad Norm 21.3184(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0007 | Time 84.4875, Epoch Time 1703.4323(1756.2369), Bit/dim 4.9840(best: 4.8040), Bit/dim Marginal 4.9171(best: 4.7534), Xent 2.1525, Loss 6.0602, Error 0.7451(best: 0.6562)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0720 | Time 15.4713(15.0141) | Bit/dim 4.6709(4.6700) | Xent 1.5699(1.4789) | Loss 12.5907(13.1642) | Error 0.5422(0.5179) Steps 610(582.96) | Grad Norm 43.1206(24.0577) | Total Time 0.00(0.00)
Iter 0730 | Time 15.0308(15.0747) | Bit/dim 4.6358(4.6661) | Xent 1.4427(1.4952) | Loss 12.6077(13.0103) | Error 0.5044(0.5217) Steps 634(588.04) | Grad Norm 14.5210(25.3713) | Total Time 0.00(0.00)
Iter 0740 | Time 14.8870(15.1137) | Bit/dim 4.5846(4.6529) | Xent 1.3789(1.4907) | Loss 12.0984(12.8362) | Error 0.4800(0.5220) Steps 562(586.24) | Grad Norm 6.7961(23.4742) | Total Time 0.00(0.00)
Iter 0750 | Time 14.9867(15.1259) | Bit/dim 4.5859(4.6348) | Xent 1.4621(1.4697) | Loss 12.1405(12.6779) | Error 0.5000(0.5140) Steps 574(584.63) | Grad Norm 30.9731(2